# Predicting Heart Disease Using Clinical Variables

In [1]:
import pandas as pd
import numpy as np

from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix,classification_report

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

from imblearn.over_sampling import SMOTE

In [2]:
df=pd.read_csv("Heart_Disease_Prediction.csv") #https://www.kaggle.com/datasets/thedevastator/predicting-heart-disease-risk-using-clinical-var

### EDA

In [3]:
df

,index,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,Absence
266,266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,Absence
267,267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,Absence
268,268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,Absence


In [4]:
df.isnull().sum() #We examine the empty lines.

index                      0
Age                        0
Sex                        0
Chest pain type            0
BP                         0
Cholesterol                0
FBS over 120               0
EKG results                0
Max HR                     0
Exercise angina            0
ST depression              0
Slope of ST                0
Number of vessels fluro    0
Thallium                   0
Heart Disease              0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    270 non-null    int64  
 1   Age                      270 non-null    int64  
 2   Sex                      270 non-null    int64  
 3   Chest pain type          270 non-null    int64  
 4   BP                       270 non-null    int64  
 5   Cholesterol              270 non-null    int64  
 6   FBS over 120             270 non-null    int64  
 7   EKG results              270 non-null    int64  
 8   Max HR                   270 non-null    int64  
 9   Exercise angina          270 non-null    int64  
 10  ST depression            270 non-null    float64
 11  Slope of ST              270 non-null    int64  
 12  Number of vessels fluro  270 non-null    int64  
 13  Thallium                 270 non-null    int64  
 14  Heart Disease            2

In [6]:
del df["index"] #We delete the index column.

In [7]:
df["Heart Disease"].value_counts()

Absence     150
Presence    120
Name: Heart Disease, dtype: int64

In [8]:
df["Heart Disease"]=df["Heart Disease"].map({"Absence":0,"Presence":1})
#We replace the data in Heart Disease with 0 and 1.
df['Heart Disease']=df['Heart Disease'].astype(float) #We change the data type of Heart Disease to float.

In [9]:
abs(df.corr()['Heart Disease']).sort_values(ascending=False) #We look at their correlations.

Heart Disease              1.000000
Thallium                   0.525020
Number of vessels fluro    0.455336
Exercise angina            0.419303
Max HR                     0.418514
ST depression              0.417967
Chest pain type            0.417436
Slope of ST                0.337616
Sex                        0.297721
Age                        0.212322
EKG results                0.182091
BP                         0.155383
Cholesterol                0.118021
FBS over 120               0.016319
Name: Heart Disease, dtype: float64

### Classification

In [48]:
x,y=df.drop(["Heart Disease","FBS over 120","EKG results","BP","Cholesterol"],axis=1),df[["Heart Disease"]]
smote = SMOTE(sampling_strategy='minority')
x_sm, y_sm = smote.fit_resample(x, y)
x_sm=scaler.fit_transform(x_sm)

In [49]:
x_train,x_test,y_train,y_test=train_test_split(x_sm,y_sm,test_size=0.20,random_state=42)

In [50]:
x.shape

(270, 9)

In [51]:
model=Sequential()
model.add(Dense(12,activation="relu"))
model.add(Dense(24,activation="relu"))
model.add(Dense(48,activation="relu"))
model.add(Dense(24,activation="relu"))
model.add(Dense(6,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics="accuracy")

In [52]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=10,epochs=50,verbose=1)
model.summary()

Epoch 1/50
24/24 [==============================] - 1s 14ms/step - loss: 0.6817 - accuracy: 0.6125 - val_loss: 0.6768 - val_accuracy: 0.5833
Epoch 2/50
24/24 [==============================] - 0s 2ms/step - loss: 0.6587 - accuracy: 0.6875 - val_loss: 0.6473 - val_accuracy: 0.7000
Epoch 3/50
24/24 [==============================] - 0s 2ms/step - loss: 0.6192 - accuracy: 0.7458 - val_loss: 0.6021 - val_accuracy: 0.7167
Epoch 4/50
24/24 [==============================] - 0s 2ms/step - loss: 0.5631 - accuracy: 0.7708 - val_loss: 0.5416 - val_accuracy: 0.7167
Epoch 5/50
24/24 [==============================] - 0s 2ms/step - loss: 0.5111 - accuracy: 0.7750 - val_loss: 0.5175 - val_accuracy: 0.7333
Epoch 6/50
24/24 [==============================] - 0s 2ms/step - loss: 0.4894 - accuracy: 0.7625 - val_loss: 0.4381 - val_accuracy: 0.8000
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 0.4603 - accuracy: 0.7833 - val_loss: 0.4422 - val_accuracy: 0.7833
Epoch 8/50
24/24 [=

In [53]:
scores=model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.3917 - accuracy: 0.8500
